# EEU4C16/EEP5C16 Lab 7 - Fast Single Image Super-Resolution

## Background
Super-resolution is a technique in image and video processing whereby a low resolution (LR) image/video frame is upsampled to a higher resolution (HR). This has numerous applications for:

- Balancing image/video quality and transmission efficiency for websites.

- Upscaling of photos taken on mobile devices ([Google Research Blog - Enhance! RAISR Sharp Images with Machine Learning](https://research.google/blog/enhance-raisr-sharp-images-with-machine-learning/)).

- Upscaling of video games ([NVIDIA - DLSS 4](https://www.nvidia.com/en-us/geforce/technologies/dlss/)).

Fast super-resolution is of utmost importance for real-time applications and a number of companies are looking into this: [Qualcomm - QuickSRNet: Plain Single-Image Super-Resolution Architecture for Faster Inference on Mobile Platforms](https://openaccess.thecvf.com/content/CVPR2023W/MobileAI/papers/Berger_QuickSRNet_Plain_Single-Image_Super-Resolution_Architecture_for_Faster_Inference_on_Mobile_CVPRW_2023_paper.pdf)

## Your Task
Train a super-resolution model to up-sample image patches from $32\times32$ to $128\times128$ (a factor of $\times4$) and get as close to real-time processing as possible (take this to mean processing a $30$ $fps$ video stream which means your model inference time should be close to $1/30$ seconds $~0.03$ $s$).

**Data Pre-Processing Requirements:**

1. You will be given a training dataset that is represenatitve of the target upsampling distribution (you may want to augment this with other datasets you can find online).

2. You will have to downsample these images and use them as input to the super-resolution neural network.

3. Generate an approriate training/test split from your dataset.

**Architecture Requirements:**

1. Design and train a deep neural network architecture that finds a tradeoff between speed and accuracy.

2. You will need to implement the model class (`SuperResolutionModel`) in (`models.py`) and save the model weights as `model.pth` (**these class and filenames must be left unchanged**).

Finally you will create a report on your data processing/network design process. Try and create some nice informative graphs, images and tables for this. 

## Important Notes (Read Carefully!)
- The evaluation on the server will record measures of [PSNR](https://en.wikipedia.org/wiki/Peak_signal-to-noise_ratio), [SSIM](https://en.wikipedia.org/wiki/Structural_similarity_index_measure), as well as parameter count (should be **below 5 million params**). You should talk about efficiency runtime etc. in your report. Note that you are free to consider other image quality metrics. You can do some research about these other metrics and consider them for discussion in your report. You can also consider loss functions that go beyond the simple $L_1$/$L_2$ losses.

- Note that the human visual system is more sensitive to changes in luminance
  than chrominance ([Wikipedia - Luma
  (video)](https://en.wikipedia.org/wiki/Luma_(video))). You can take advantage
  of this.

- If you choose to use you own dataset you must save it to a directory on the
  Colab VM as we have done in the cell below i.e.
  `/home/tcd/super_resolution/dataset.npz`, **DO NOT** save it anywhere in the
  `4c16-labs/code` directory as it will affect the whole git system. 

- If you are making your own dataset, please chose seperate temporory working
  directory outside `4c16-labs/code`. You could make as `4c16-labs/dataset` to
  store. You CANNOT train and use dataset directly from Google Drive, you SHOULD
  move to Google Colab VM dataset (eg. `home/tcd/super_resolution` ) every session. 

- There are different conventions for specifying the dimensions of input/output tensors i.e. $(B, H, W, C)$ or $(B, C, H, W)$ where:    
    - $B=$ batch index
    - $H=$ image height
    - $W=$ image width
    - $C=$ image channels (generally $3$ e.g. $RGB$, $YUV$) 

- We expect your network to have $(B, C, H, W)$ (channel first convention used by torch layers) as the input and output tensor dimensions

- Your score for this lab is not directly tied to your performance on the backend. It will be a combination of your network performance and how you present your design in the interview.

In [ ]:
# This cell mounts Colab to your Google Drive and navigates to the script directory
from google.colab import drive
drive.mount('/content/gdrive')
%cd /content/gdrive/MyDrive/4c16-labs/code/lab-07/

In [ ]:
# Create a .gitignore file to avoid committing large files to git accidentlly
# You should run this cell only once at start of the lab.
# c.f. https://git-scm.com/docs/gitignore
%%writefile .gitignore
*.pyc
__pycache__/
.png
.jpg
.zip
.tar
.tar*

In [ ]:
# Download the dataset (should take about ~2 minutes, 617MB)
!curl --create-dirs -o /home/tcd/super_resolution/dataset.npz https://tcddeeplearning.blob.core.windows.net/deeplearning202324/hr_images.npz

In [ ]:
# Import necessary functions libraries
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

# Import our model definition (if you get a module import error make sure your
# current working directory is the script directory is 4c16-labs/code/lab-07/
# from your Google Drive
from models import SuperResolutionModel

# Dataset pre-processing
Here we load the dataset we have provided from a Numpy compressed (`.npz`) file. It's up to you to pre-process the dataset.

In [ ]:
hr_images = np.load(f'/home/tcd/super_resolution/dataset.npz')['hr_images']
# Your input/output images should have pixel values in the range [0, 1]
# For 8-bit images the max. pixel value is 255
hr_images = hr_images / 255.0
hr_images = hr_images.astype(np.float32)

# Downsample each image with something like skimage.transform rescale function (https://scikit-image.org/docs/0.25.x/api/skimage.transform.html#skimage.transform.rescale)
#   This would be something like:
#   lr_images = rescale(hr_images, scale_factor, etc..))
# Then pair each high-res (grounn truth) with it's downsampled low-res counterpart

# Defining the Dataset Generator and Dataloader
Next we define the dataloader which will give us an iterable object of high-resolution and low-resolution image pairs to use in our training loop. You can define data augmentation strategies to randomly flip image pairs, shift pixel values etc.

In [ ]:
class SuperResolutionDataset(Dataset):
    def __init__(self, npz_file_path):
        """Load tensors from .npz file. or whatever way you choose"""
        data = torch.load(npz_file_path)
        self.hr_images = data['hr_images']
        self.lr_images = data['lr_images']

    def __len__(self):
        return len(self.hr_images)

    def __getitem__(self, idx):
        return self.lr_images[idx], self.hr_images[idx]

training_dataset = SuperResolutionDataset('/home/tcd/super_resolution/hr_lr_pairs.npz')
training_dataloader = DataLoader(training_dataset, batch_size=32, shuffle=True)

# Next steps
*   Set up your DataLoader etc. and think about augmentations, if you want to gather more datasets for training etc.
*   Set up your model architecture and training loop to see how your initial design performs and iterate on this to increase performance.
*   You should think about how you are going to plot results and show network performance in your report.

In [ ]:
sr_model = SuperResolutionModel()
criterion = nn.MSELoss() # Define your loss here
optimizer = optim.Adam(sr_model.parameters(), lr=0.001) # Choose an optimiser

# Move model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
sr_model.to(device)

num_epochs = 10

# Define your training loop
for epoch in range(num_epochs):
    sr_model.train()  # Set model to training mode
    total_loss = 0
    
    for batch_idx, (lr_images, hr_images) in enumerate(training_dataloader):
    # Do training...

# IMPORTANT
Make sure your model definition in `models.py` has the class name
`SuperResolutionModel` and make sure you save your weights as `model.pth` e.g.
`torch.save(model, 'model.pth')`. 

**DO NOT** submit a model which has more than 5 Million parameters. We will
reject in the backend. 

In [ ]:
def count_model_parameters(model):
    return sum(p.numel() for p in model.parameters())

total_params = count_model_parameters(sr_model)
if total_params > 5000000:
    print("Model greater than 5 million params!! Reduce model complexity!!")
else:
    torch.save(sr_model, 'model.pth')